In [1]:
# Import dependencies

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2

In [4]:
# Load the cleaned NYC dogs dataset and convert to Dataframe
dog_data_file_path = "../Resources/NYC_dogs_clean_forclass.csv"
NYC_dog_data = pd.read_csv(dog_data_file_path)
NYC_dog_data_df = pd.DataFrame(NYC_dog_data)
NYC_dog_data_df.head()

,UID,AnimalName,AnimalGender,AnimalBirthMonth,BreedName,Borough,ZipCode,CommunityDistrict,CensusTract,NTA,CityCouncilDistrict,CongressionalDistrict,StateSenatorialDistrict,LicenseIssuedDate,LicenseExpiredDate
0,0,SHADOW,M,1,Beagle,Brooklyn,11236,318.0,1014.0,BK50,46.0,8.0,19.0,12/29/14,1/30/16
1,1,ROCCO,M,10,Boxer,Brooklyn,11210,314.0,756.0,BK43,45.0,9.0,17.0,1/7/15,1/30/16
2,2,LUIGI,M,9,Maltese,Bronx,10464,210.0,516.0,BX10,13.0,14.0,34.0,1/17/15,2/2/16
3,3,PETUNIA,F,8,Pug,Brooklyn,11221,304.0,419.0,BK78,34.0,7.0,18.0,3/1/15,3/28/16
4,4,ROMEO,M,10,Maltese,Bronx,10451,201.0,65.0,BX34,17.0,15.0,32.0,3/9/15,3/9/16


In [33]:
NYC_dog_data_cleaned_df = NYC_dog_data_df.filter(['UID','Borough',
                                                  'AnimalName',
                                                  'AnimalGender', 
                                                  'AnimalBirthMonth',
                                                  'BreedName', 
                                                  'CensusTract',
                                                 'LicenseIssuedDate',
                                                 'LicenseExpiredDate'], axis=1).rename(columns={'AnimalName':'Name','AnimalGender':'Gender', 
                                        'AnimalBirthMonth': 'BirthMonth','LicenseIssuedDate':'IssueDate','LicenseExpiredDate':'ExpirationDate'
                                        })
NYC_dog_data_cleaned_df.tail()

,UID,Borough,Name,Gender,BirthMonth,BreedName,CensusTract,IssueDate,ExpirationDate
117619,121857,Manhattan,ROCKY,M,6,Boston Terrier,98.0,12/31/16,12/31/18
117620,121858,Brooklyn,ROXY,F,10,"Collie, Border",250.0,12/31/16,9/18/18
117621,121859,Staten Island,VALENTINA,F,5,German Shepherd Crossbreed,17008.0,12/31/16,12/31/17
117622,121860,Bronx,VENUS,F,12,Yorkshire Terrier,79.0,12/31/16,6/6/17
117623,121861,Manhattan,WILLOW,F,3,Unknown,195.0,12/31/16,1/15/18


In [34]:
NYC_dog_data_cleaned_df.dtypes

UID                 int64
Borough            object
Name               object
Gender             object
BirthMonth          int64
BreedName          object
CensusTract       float64
IssueDate          object
ExpirationDate     object
dtype: object

In [9]:
income_data_file_path = "../Resources/income_by_year.csv"
NYC_income_data_cleaned = pd.read_csv(income_data_file_path)
NYC_income_data_df = pd.DataFrame(NYC_income_data_cleaned)
NYC_income_data_df.tail()

,Tract,AvgHI2014,AvgHI2015,AvgHi2016,AverageHI
1308,1579.01,78641.0,82981.0,93466.0,85029.33
1309,1579.02,89395.0,95592.0,103125.0,96037.33
1310,1579.03,81173.0,76330.0,82148.0,79883.67
1311,1617.00,88125.0,73929.0,76045.0,79366.33
1312,1621.00,74861.0,67425.0,65114.0,69133.33


In [16]:
# Round census tract column of the income data DataFrame to 1 decimal place to match NYC dog data DataFrame
NYC_income_data_df['Tract'] = NYC_income_data_df['Tract'].round(decimals=1)

NYC_income_data_df.head()

,Tract,AvgHI2014,AvgHI2015,AvgHi2016,AverageHI
0,1.0,102825.0,99372.50,108892.0,103696.50
1,2.0,57469.0,57005.33,62204.0,58892.78
2,2.0,23036.0,20521.00,21102.0,21553.00
3,2.0,29418.0,29684.00,32411.0,30504.33
4,3.0,57500.0,59688.00,70078.0,62422.00


In [12]:
# Retrieve the (cleaned) length of NYC_dog_data_df
len(NYC_dog_data_df)

117624

In [21]:
# Find the total number of adopted animals in NYC boroughs
NYC_dog_data_cleaned_df.groupby('Borough')['UID'].count()

Borough
Bronx            12328
Brooklyn         28950
Manhattan        41327
Queens           24154
Staten Island    10865
Name: UID, dtype: int64

In [22]:
# initialize lists 
data = [['Bronx', 12328], ['Brooklyn', 28950], ['Manhattan', 41288], ['Queens', 41327], ['Staten Island', 10865]]
  
# Create a new DataFrame for total dog population in each NYC borough
NYC_dog_pop_df = pd.DataFrame(data, columns = ['borough', 'dog_population'])

NYC_dog_pop_df

,borough,dog_population
0,Bronx,12328
1,Brooklyn,28950
2,Manhattan,41288
3,Queens,41327
4,Staten Island,10865


In [23]:
# Create a new df of dog gender and boroughs
gender_boroughs_df = NYC_dog_data_cleaned_df.filter(['UID','Borough','Gender'], axis=1)
gender_boroughs_df

,UID,Borough,Gender
0,0,Brooklyn,M
1,1,Brooklyn,M
2,2,Bronx,M
3,3,Brooklyn,F
4,4,Bronx,M
...,...,...,...
117619,121857,Manhattan,M
117620,121858,Brooklyn,F
117621,121859,Staten Island,F
117622,121860,Bronx,F


In [25]:
# Find the number of male and female dogs adopted in NYC
gender_boroughs_df.groupby(['Borough', 'Gender'])['Gender'].count()

Borough        Gender
Bronx          F          5391
               M          6937
Brooklyn       F         13166
               M         15784
Manhattan      F         19247
               M         22080
Queens         F         10675
               M         13479
Staten Island  F          4968
               M          5897
Name: Gender, dtype: int64

In [26]:
# Create a dataframe based off dog birth months
birth_month_boroughs_df = NYC_dog_data_cleaned_df.filter(['UID','Borough','BirthMonth'], axis=1)


birth_month_boroughs_df

,UID,Borough,BirthMonth
0,0,Brooklyn,1
1,1,Brooklyn,10
2,2,Bronx,9
3,3,Brooklyn,8
4,4,Bronx,10
...,...,...,...
117619,121857,Manhattan,6
117620,121858,Brooklyn,10
117621,121859,Staten Island,5
117622,121860,Bronx,12


In [28]:
# Create a dataframe based off dog birth months
breeds_boroughs_df = NYC_dog_data_cleaned_df.filter(['UID','Borough','BreedName'], axis=1)

breeds_boroughs_df

,UID,Borough,BreedName
0,0,Brooklyn,Beagle
1,1,Brooklyn,Boxer
2,2,Bronx,Maltese
3,3,Brooklyn,Pug
4,4,Bronx,Maltese
...,...,...,...
117619,121857,Manhattan,Boston Terrier
117620,121858,Brooklyn,"Collie, Border"
117621,121859,Staten Island,German Shepherd Crossbreed
117622,121860,Bronx,Yorkshire Terrier


In [29]:
# Find the top breeds adopted in NYC boroughs
breeds_boroughs_df['BreedName'].value_counts()

Unknown                    16419
Yorkshire Terrier           7358
Shih Tzu                    6848
Chihuahua                   5554
Labrador Retriever          4135
                           ...  
Pyrenean Shepherd              1
Afghan Hound Crossbreed        1
Finnish Lapphund               1
Swedish Vallhund               1
Neapolitan Mastiff             1
Name: BreedName, Length: 299, dtype: int64

In [30]:
# Create a dataframe based off dog birth months
names_boroughs_df = NYC_dog_data_cleaned_df.filter(['UID','Borough','Name'], axis=1)

names_boroughs_df

,UID,Borough,Name
0,0,Brooklyn,SHADOW
1,1,Brooklyn,ROCCO
2,2,Bronx,LUIGI
3,3,Brooklyn,PETUNIA
4,4,Bronx,ROMEO
...,...,...,...
117619,121857,Manhattan,ROCKY
117620,121858,Brooklyn,ROXY
117621,121859,Staten Island,VALENTINA
117622,121860,Bronx,VENUS


In [32]:
# Find the top names given to adopted dogs in NYC boroughs
names_boroughs_df['Name'].value_counts()

BELLA        1008
MAX          1000
CHARLIE       750
LOLA          683
ROCKY         667
             ... 
ICEMILLER       1
SADIRA          1
SEERA           1
SHOBE           1
KYANDI          1
Name: Name, Length: 20726, dtype: int64

In [36]:
# Create a dataframe based off license issue and expiration dates
license_dates_df = NYC_dog_data_cleaned_df.filter(['UID','Borough','IssueDate','ExpirationDate'], axis=1)

license_dates_df

,UID,Borough,IssueDate,ExpirationDate
0,0,Brooklyn,12/29/14,1/30/16
1,1,Brooklyn,1/7/15,1/30/16
2,2,Bronx,1/17/15,2/2/16
3,3,Brooklyn,3/1/15,3/28/16
4,4,Bronx,3/9/15,3/9/16
...,...,...,...,...
117619,121857,Manhattan,12/31/16,12/31/18
117620,121858,Brooklyn,12/31/16,9/18/18
117621,121859,Staten Island,12/31/16,12/31/17
117622,121860,Bronx,12/31/16,6/6/17


In [37]:
# Connect to postgres/pgAdmin
sql_pgadmin = psycopg2.connect(user="postgres", password="", host="localhost", port="5432", database="dog_licenses_db")


In [38]:
# Convert DataFrames into csvs

NYC_dog_data_cleaned_df.to_csv('../Resources/cleaned_NYC_dog_data.csv', index=False)

NYC_dog_pop_df.to_csv('../Resources/NYC_dog_pop_df.csv', index=False)

gender_boroughs_df.to_csv('../Resources/NYC_dog_genders.csv', index=False)

birth_month_boroughs_df.to_csv('../Resources/NYC_dog_birth_months.csv', index=False)

breeds_boroughs_df.to_csv('../Resources/NYC_breed_names.csv', index=False)

names_boroughs_df.to_csv('../Resources/NYC_dog_names.csv', index=False)

license_dates_df.to_csv('../Resources/license_dates.csv', index=False)

In [40]:
# Test connection to database
query = "SELECT * FROM NYC_dog_data;"
example_df = pd.read_sql(query, sql_pgadmin)
example_df

,uid,borough,name,gender,birthmonth,breedname,censustract,issuedate,expirationdate
0,0,Brooklyn,SHADOW,M,1,Beagle,1014.0,2014-12-29,2016-01-30
1,1,Brooklyn,ROCCO,M,10,Boxer,756.0,2015-01-07,2016-01-30
2,2,Bronx,LUIGI,M,9,Maltese,516.0,2015-01-17,2016-02-02
3,3,Brooklyn,PETUNIA,F,8,Pug,419.0,2015-03-01,2016-03-28
4,4,Bronx,ROMEO,M,10,Maltese,65.0,2015-03-09,2016-03-09
...,...,...,...,...,...,...,...,...,...
117619,121857,Manhattan,ROCKY,M,6,Boston Terrier,98.0,2016-12-31,2018-12-31
117620,121858,Brooklyn,ROXY,F,10,"Collie, Border",250.0,2016-12-31,2018-09-18
117621,121859,Staten Island,VALENTINA,F,5,German Shepherd Crossbreed,17008.0,2016-12-31,2017-12-31
117622,121860,Bronx,VENUS,F,12,Yorkshire Terrier,79.0,2016-12-31,2017-06-06
